In [3]:
import matplotlib.pyplot as plt
import importlib.util
import pandas as pd
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib
import os
from matplotlib.colors import LogNorm
from matplotlib.colors import Normalize

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [4]:
def get_bin_widths():
    h = 6.62607004e-34
    e = 1.60217662e-19
    freq_path = '/ptmp/mpa/mglatzle/TNG_f_esc/fid2/BB_T1d4_L2d38_r67_DUST.ps'
    freq = np.loadtxt(freq_path, skiprows=1)[1:,0]
    bin_widths = []
    centroids = []
    for i in range(len(freq)-1):
        bin_widths.append(freq[i+1]-freq[i])
        centroids.append(freq[i]/2+freq[i+1]/2)
    energies = np.array(centroids)*h/e    
    return (np.array(bin_widths)*h/e, energies)

In [5]:
def get_tot_photons(df,IDs=None,emitted=True):
    if IDs==None:
        IDs = df.index.values
    tot_photons = np.zeros(df.loc[IDs[0]]['emitted_photons'].shape[0])
    tot_norm = np.zeros(df.loc[IDs[0]]['emitted_photons'].shape[0])
    Q0_sum = 0
    for ID in IDs:
        if emitted:
            photons = df.loc[ID]['Q0']*df.loc[ID]['emitted_photons']/df.loc[ID]['n_iterations']
            norm = photons
        else:
            norm = df.loc[ID]['Q0']*df.loc[ID]['emitted_photons']/df.loc[ID]['n_iterations']
            photons = df.loc[ID]['Q0']*df.loc[ID]['escaped_photons']/df.loc[ID]['n_iterations']
        try:
            photons_cum = photons.sum(axis=1)
            norm_cum = norm.sum(axis=1)
        except:
#             print(photons)
#             print(ID)
            continue
        if (np.isnan(np.sum(photons_cum)) or np.isnan(np.sum(norm_cum))):
            continue
        else:
            tot_photons += photons_cum
            tot_norm += norm_cum
            #print(np.sum(norm_cum))
            Q0_sum += df.loc[ID]['Q0']
    return tot_photons/Q0_sum, tot_norm/Q0_sum

In [8]:
def get_photon_spectrum(df, IDs=None, emitted=True):
    if IDs==None:
        IDs = df.index.values
    tot_photons = np.zeros(df.loc[IDs[0]]['emitted_photons'].shape[0])
    tot_photons2 = np.zeros(df.loc[IDs[0]]['emitted_photons'].shape[0])
    
    Q0_sum = 0
    spec_counter = 0
    for ID in IDs:
        try:
            if emitted:
                photons = df.loc[ID]['Q0']*df.loc[ID]['emitted_photons'].sum(axis=1)/df.loc[ID]['n_iterations']
                photons2 = df.loc[ID]['Q0']*(df.loc[ID]['emitted_photons'].sum(axis=1)/df.loc[ID]['n_iterations'])**2
            else:
                photons = df.loc[ID]['Q0']*df.loc[ID]['escaped_photons'].sum(axis=1)/df.loc[ID]['n_iterations']
                photons2 = df.loc[ID]['Q0']*(df.loc[ID]['escaped_photons'].sum(axis=1)/df.loc[ID]['n_iterations'])**2
        except:
            continue

        if np.isnan(np.sum(photons)):
            continue
        else:
            tot_photons += photons
            tot_photons2 += photons2
            Q0_sum += df.loc[ID]['Q0']
            spec_counter += 1
            
    average_p = tot_photons/Q0_sum
    error_p = np.sqrt(tot_photons2/Q0_sum-average_p**2)/np.sqrt(spec_counter)
    return average_p, error_p

In [9]:
def get_spectrum(df, IDs=None, emitted=True):
    bin_widths, energies = get_bin_widths()
    average_p, error = get_photon_spectrum(df, IDs, emitted)
    norm = average_p.sum()
    spectrum = average_p/norm/bin_widths
    error_norm = error/norm/bin_widths
    return (energies, bin_widths, spectrum, error_norm)

### Function which plots the average or median frequency dependent escape fraction:
- **dataframes**: List of dataframes for which the escape fraction needs to be analysed
- **mode**: Do you want to take the mean or the average over the halo escape fractions
- **labels**: Labels for the graphs, by default they will be named 'dataframe n' where n is the number of the dataframe as it appears in the dataframes list

In [10]:
def freq_fesc(dataframes, mode='mean', labels=None, savename=None, residual=False, save=False):
    plt.rcParams["figure.figsize"] = (30,20)
    
    linewidth = 6
    vert_width = 3
    labelsize = 95
    axes_width = 3
    ticksize = 70
    major_tick_length = 16
    major_tick_width = 4
    minor_tick_length = 8
    minor_tick_width = 3
    legendsize = 50
    hI_ion = 13.6
    heI_ion = 24.6
    heII_ion = 54.4
    vert_width=3
    x_label = r'$E[\mathrm{eV}]$'
    
    if residual:
        y_label = r'$\frac{\langle f_{\mathrm{esc, dust}} \rangle}{\langle f_{\mathrm{esc, no \ dust}} \rangle}$'
    else:
        y_label = r'$\langle f_{\mathrm{esc}} \rangle$'
    #Set natural constants to convert photon frequency to energy in eV
    h = 6.62607004e-34
    e = 1.60217662e-19
    if labels == None:
        labels=[]
        for i in range(len(dataframes)):
            labels.append(f'dataframe {i+1}')

    values_array = []
    error_array = []
    for i, dataframe in enumerate(dataframes):
        energies = dataframe.iloc[0]['frequencies']*h/e

        per_freq_data = np.vstack(dataframe.loc[:,'per_freq'].to_numpy())
        error = np.std(per_freq_data, axis=0)/np.sqrt(len(per_freq_data))

        if mode == 'mean':
            values = np.mean(per_freq_data, axis=0)
        elif mode == 'median':
            values = np.median(per_freq_data, axis=0)
        else:
            raise ValueError('The possible modes are "mean" and "median"')
        #plt.ylim(0,0.35)
        if residual:
            values_array.append(values)
            error_array.append(error)
        else:
            plt.errorbar(energies, values, yerr=error, linewidth=5, elinewidth=1, capsize=10, label=str(labels[i]))
    
    if residual:
        error = error_array[1]/values_array[0]+values_array[1]*error_array[0]/(values_array[0]**2)
        plt.errorbar(energies, values_array[1]/values_array[0], yerr=error, linewidth=7, elinewidth=1, capsize=10, color='black')
        plt.ylim(0.7,1.23)
    
    vert_color='black' 
    vert_style = '--'
    plt.axvline(x=hI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    plt.axvline(x=heI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    plt.axvline(x=heII_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    plt.xlabel(x_label, fontsize = labelsize)
    plt.ylabel(y_label, fontsize = labelsize)
    
    plt.rc('axes', linewidth=axes_width)
    plt.rc('xtick', labelsize=ticksize) 
    plt.rc('ytick', labelsize=ticksize) 
    plt.tick_params(length=major_tick_length, width=major_tick_width)
    plt.tick_params(length=minor_tick_length, width=minor_tick_width, which='minor')
    plt.legend(fontsize=legendsize)
    if save:
        if savename == None:
            plt.savefig(f'/u/ivkos/analysis/plots/freq_fesc.pdf', bbox_inches = 'tight', pad_inches = 0)
        else:
            plt.savefig(f'/u/ivkos/analysis/plots/{savename}.pdf', bbox_inches = 'tight', pad_inches = 0)
    plt.show()
    return 

In [11]:
def get_wavelength(energy):
    h = 6.62607004e-34
    e = 1.60217662e-19
    c = 299792458.
    to_ang = 1e10
    return h*c/energy/e*to_ang

def get_energy(wavelength):
    h = 6.62607004e-34
    e = 1.60217662e-19
    c = 299792458.
    to_ang = 1e10
    return h*c/wavelength/e/to_ang

In [12]:
def plot_multiple_spectra_comparison(dfs_array, colors=None, labels=None, tag='', save=False, top_title=False):
    
    hI_ion = 13.6
    heI_ion = 24.6
    heII_ion = 54.4
    
    labelsize = 95
    legendsize = 57
    legend_loc = 'lower left'
    axis_width = 4
    vert_width = 2
    vert_color='black' 
    vert_style = '--'
    tick_label_size = 75
    figsize_x = 30
    figsize_y = 70
    ticks = [20, 50, 100]
    ticks_upper = [1e3, 3e2, 1e2]
    x_axis_label = '$E_\gamma[\mathrm{eV}]$'
    y_axis_label = '$P(E)$'
    sec_x_axis_label = r'$\lambda [\mathrm{\AA}]$'
    x_tick_major_size = 32
    x_tick_major_width = 8
    x_tick_minor_size = 16
    x_tick_minor_width = 4
    y_lim_low = 2e-7
    y_lim_high = 2e-1
    capsize=6 
    
    plt.rc('xtick', labelsize=tick_label_size) 
    plt.rc('ytick', labelsize=tick_label_size) 
    
    if colors==None:
        colors=['b','r','g', 'c','m','y']
    if labels==None:
        labels=[x for x in range(len(dfs_array[0]))]
        
    f = plt.figure()
    plt.subplots_adjust(hspace=0.001)
    plt.subplots_adjust(wspace=0.001)
    
    ax1 = plt.subplot2grid((3, 1), (0,0))
    energies, widths, spec_em, error_em = get_spectrum(dfs_array[0], IDs=None, emitted=True)
    energies, widths, spec_es, error_es = get_spectrum(dfs_array[0], IDs=None, emitted=False)

    ax1.errorbar(energies, spec_em, yerr=error_em, capsize=capsize, label=f'{labels[0]}', linewidth=3, color=colors[0])
    ax1.errorbar(energies, spec_es, yerr=error_es, capsize=capsize, label=f'{labels[1]}', linewidth=3, color=colors[0], linestyle='dashed')

    ax1.axvline(x=hI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax1.axvline(x=heI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax1.axvline(x=heII_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax1.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
        
    ax2 = plt.subplot2grid((3, 1), (1,0), sharex=ax1)
    for i, df in enumerate(dfs_array[1]):
        energies, widths, spec_em, error_em = get_spectrum(df, IDs=None, emitted=True)
        energies, widths, spec_es, error_es = get_spectrum(df, IDs=None, emitted=False)

        ax2.errorbar(energies, spec_em, yerr=error_em, capsize=capsize, label=f'{labels[i+2]} emitted', linewidth=3, color=colors[i])
        ax2.errorbar(energies, spec_es, yerr=error_es, capsize=capsize, label=f'{labels[i+2]} escaped', linewidth=3, color=colors[i], linestyle='dashed')

    ax2.axvline(x=hI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax2.axvline(x=heI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax2.axvline(x=heII_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax2.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    
    ax3 = plt.subplot2grid((3, 1), (2,0), sharex=ax2)
    for i, df in enumerate(dfs_array[2]):
        energies, widths, spec_em, error_em = get_spectrum(df, IDs=None, emitted=True)
        energies, widths, spec_es, error_es = get_spectrum(df, IDs=None, emitted=False)

        ax3.errorbar(energies, spec_em, yerr=error_em, capsize=capsize, label=f'{labels[i+4]} emitted', linewidth=3, color=colors[i])
        ax3.errorbar(energies, spec_es, yerr=error_es, capsize=capsize, label=f'{labels[i+4]} escaped', linewidth=3, color=colors[i], linestyle='dashed')

    ax3.axvline(x=hI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax3.axvline(x=heI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax3.axvline(x=heII_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax3.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    
    secax = ax1.secondary_xaxis('top', functions=(get_wavelength, get_energy))
    if top_title:
        secax.set_xlabel(sec_x_axis_label, fontsize=labelsize)
    secax.tick_params(length=x_tick_major_size, width=x_tick_major_width)
    secax.tick_params(length=x_tick_minor_size, width=x_tick_minor_width, which='minor')

    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    ax3.set_yscale('log')
    ax1.tick_params(length=x_tick_major_size, width=x_tick_major_width)
    ax1.tick_params(length=x_tick_minor_size, width=x_tick_minor_width, which='minor')
    ax2.tick_params(length=x_tick_major_size, width=x_tick_major_width)
    ax2.tick_params(length=x_tick_minor_size, width=x_tick_minor_width, which='minor')
    ax3.tick_params(length=x_tick_major_size, width=x_tick_major_width)
    ax3.tick_params(length=x_tick_minor_size, width=x_tick_minor_width, which='minor')
    
    ax3.tick_params(length=x_tick_major_size, width=x_tick_major_width)
    ax3.tick_params(length=x_tick_minor_size, width=x_tick_minor_width, which='minor')
    ax1.set_xticks(ticks)
    ax3.set_xticklabels(ticks)

    ax3.set_xlabel(x_axis_label, fontsize=labelsize)
    ax1.set_ylabel(y_axis_label, fontsize=labelsize)
    ax2.set_ylabel(y_axis_label, fontsize=labelsize)
    ax3.set_ylabel(y_axis_label, fontsize=labelsize)
    ax1.legend(fontsize=legendsize, loc=legend_loc)
    ax2.legend(fontsize=legendsize, loc=legend_loc)
    ax3.legend(fontsize=legendsize, loc=legend_loc)
    
    ax1.set_ylim(y_lim_low, y_lim_high)
    ax2.set_ylim(y_lim_low, y_lim_high)
    ax3.set_ylim(y_lim_low, y_lim_high)
    
    plt.rcParams["figure.figsize"] = (figsize_x,figsize_y)
    plt.rc('axes', linewidth=axis_width)
#     plt.tick_params(length=tick_size_major, width=tick_width_major)
#     plt.tick_params(length=tick_size_minor, width=tick_width_minor, which='minor')
    

    
    if save:
        plt.savefig(f'/u/ivkos/analysis/plots/spectra_comparison_{tag}.pdf', bbox_inches = 'tight')

    return

In [13]:
def plot_spectra_comparison(df, mode='mean', IDs=None, save=False, top_title=False):
    energies, widths, spec_em = get_spectrum(df, IDs=IDs, emitted=True)
    energies, widths, spec_es = get_spectrum(df, IDs=IDs, emitted=False)
    
    hI_ion = 13.6
    heI_ion = 24.6
    heII_ion = 54.4
    
    labelsize = 60
    legendsize = 45
    axis_width = 4
    vert_width=3
    linewidth = 4
    tick_size_major = 16
    tick_size_minor = 8
    tick_width_major = 4
    tick_width_minor = 3
    tick_label_size = 45
    figsize_x = 30
    figsize_y = 30
    ticks = [20, 50, 100]
    ticks_upper = [1e3, 3e2, 1e2]
    
    ax = plt.gca()
    ax.plot(energies, spec_em, label='emitted spectrum', color='black', linewidth=linewidth)
    ax.plot(energies, spec_es, label='escaped spectrum', color='grey', linewidth=linewidth)

    vert_color='black' 
    vert_style = '--'
    ax.axvline(x=hI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax.axvline(x=heI_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)
    ax.axvline(x=heII_ion, linewidth=vert_width, color=vert_color, linestyle=vert_style)

    secax = ax.secondary_xaxis('top', functions=(get_wavelength, get_energy))

    if top_title:
        secax.set_xlabel(r'$\lambda [\mathrm{\AA}]$', fontsize=labelsize)
    secax.tick_params(length=tick_size_major, width=tick_width_major)
    secax.tick_params(length=tick_size_minor, width=tick_width_minor, which='minor')
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_xticks(ticks)
    ax.set_xticklabels(ticks)
    
    secax.set_xticks(ticks_upper)
    secax.set_xticklabels(ticks_upper)

    ax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    secax.get_xaxis().set_major_formatter(matplotlib.ticker.ScalarFormatter())
    plt.rc('xtick', labelsize=tick_label_size) 
    plt.rc('ytick', labelsize=tick_label_size) 
    ax.set_xlabel('$E_\gamma[\mathrm{eV}]$', fontsize=labelsize)
    ax.set_ylabel('$P(\gamma)$', fontsize=labelsize)
    ax.legend(fontsize=legendsize)
    plt.rcParams["figure.figsize"] = (figsize_x,figsize_y)
    plt.rc('axes', linewidth=axis_width)
    plt.tick_params(length=tick_size_major, width=tick_width_major)
    plt.tick_params(length=tick_size_minor, width=tick_width_minor, which='minor')
    if save:
        plt.savefig(f'/u/ivkos/analysis/plots/spectra.pdf', bbox_inches = 'tight')

    return

In [14]:
def update_df(df, h=0.6774):
    #df.set_index('ID', inplace=True)
    df['Q0/HaloMass']=df['Q0']/df['HaloMass']
    df['R_v^3/HaloMass']=df['HaloRadii']**3/df['HaloMass']
    df['StarMass']= df['HaloMass']*df['FractionStars']*1e10/h
    return

In [15]:
df = pd.read_pickle('dfs/esc_analysis.pickle')
update_df(df)

In [16]:
df_dust = pd.read_pickle('dfs/new_dust.pickle')
update_df(df_dust)

In [17]:
df_no_dust =  df.merge(df_dust[['ID','z']], on=['ID','z'], how='inner')

In [18]:
df = df.drop([6676, 6713,7507])

In [19]:
filt_f_esc = df.f_esc > 0.1
filt_f_esc_small = df.f_esc < 0.01
filt_star = df.StarMass > 5e7
filt_star_small = df.StarMass < 5e6
dfs_1=df
dfs_3=[df[filt_f_esc],df[filt_f_esc_small]]
dfs_2=[df[filt_star], df[filt_star_small]]
labels_1 = ['emitted spectrum', 'escaped spectrum', '$M_{\star}>10^8M_{\u2609}$', r'$M_{\star}<5 \times$' +  '$10^6M_{\u2609}$', r'$f_\mathrm{esc}>0.1$',r'$f_\mathrm{esc}<0.01$']
#labels_2 = [r'no dust',r'dust']
#labels_1.extend(labels_2)
tag_1 = 'esc'
tag_2 = 'mass'